In [1]:
import MySQLdb as mdb
import pandas as pd
from os import listdir

In [2]:
EEG_DATA_DIR = "/home/dannowitz/Downloads/kaggle-eeg"

In [3]:
# Get list of files, split into data.csv and events.csv files
sample_submission = "sample_submission.csv"
test_data_list = listdir(EEG_DATA_DIR + "/test")
train_list = listdir(EEG_DATA_DIR + "/train")
train_events_list = [f for f in train_list if f[-10:]=="events.csv"]
train_data_list = [f for f in train_list if f[-8:]=="data.csv"]
del train_list

In [4]:
data_table_create = """
                    CREATE TABLE IF NOT EXISTS %s (
                        id VARCHAR(32), Fp1 INT, Fp2 INT, F7 INT, 
                        F3 INT, Fz INT, F4 INT, F8 INT, FC5 INT, 
                        FC1 INT, FC2 INT, FC6 INT, T7 INT, C3 INT, 
                        Cz INT, C4 INT, T8 INT, TP9 INT, CP5 INT, 
                        CP1 INT, CP2 INT, CP6 INT, TP10 INT, P7 INT,
                        P3 INT, Pz INT, P4 INT, P8 INT, PO9 INT, 
                        O1 INT, Oz INT, O2 INT, PO10 INT)
                    """

In [5]:
event_table_create = """
                     CREATE TABLE IF NOT EXISTS %s (
                     id VARCHAR(32),
                     HandStart INT,
                     FirstDigitTouch INT,
                     BothStartLoadPhase INT,
                     LiftOff INT,
                     `Replace` INT,
                     BothReleased INT )
                     """

In [6]:
load_query = """
             LOAD DATA LOCAL INFILE '%s/%s'
             INTO TABLE %s
             FIELDS TERMINATED BY ','
             IGNORE 1 LINES
             """

In [7]:
# CHOOSE ONE SERVER TO WORK ON
server = "seaquel.physics.illinois.edu"
port = 3283
schema = "user_dannowitz_kaggle"
user = "seaguest"
password = "XXXXXXXXXX"

In [8]:
# CHOOSE ONE SERVER TO WORK ON
server = "kaggle-eeg.cginynb2jyca.us-west-2.rds.amazonaws.com"
port = 3306
schema = "eeg"
user = "quarkonia"
password = "XXXXXXXXXX"

### Load data into MySQL
* Fastest operation to get data from source to table is LOAD DATA LOCAL INFILE
* Create schema
* Create tables
* Perform LOAD DATA command on each file

In [ ]:
try:
    db = mdb.connect(user=user,
                     passwd=password,
                     host=server,
                     port=port,
                     local_infile = 1)
    
    cur = db.cursor()
    
    # See if our schema exists
    cur.execute("SHOW DATABASES LIKE '%s'" % schema)
    if cur.rowcount == 0:
        cur.execute("CREATE DATABASE %s" % schema)
    
    # Set as default
    cur.execute("USE %s" % schema)
    
    # Create and populate the test data table
    table = "test_data"
    cur.execute(data_table_create % (table))
    for data_file in test_data_list:
        cur.execute(load_query % (EEG_DATA_DIR, "test/" + data_file, table))
        
    # Create and populate the train data table
    table = "train_data"
    cur.execute(data_table_create % (table))
    for data_file in train_data_list:
        cur.execute(load_query % (EEG_DATA_DIR, "train/" + data_file, table))
    
    # Create and populate the test and train events tables
    table = "test_events"
    cur.execute(event_table_create % (table))
    table = "train_events"
    cur.execute(event_table_create % (table))
    for events_file in train_events_list:
        cur.execute(load_query % (EEG_DATA_DIR, "train/" + events_file, table))
    
    # Throw the sample submission example in the DB as well
    table = "sample_submission"
    cur.execute(event_table_create % (table))
    cur.execute(load_query % (EEG_DATA_DIR, sample_submission, table))
    
    if db:
        db.close()

except mdb.Error, e:

    print "Error %d: %s" % (e.args[0], e.args[1])

### We want to create three new fields
* Subject
* Session
* Frame
* All after 'id' field in tables

Note: You *could* just alter the existing table to have new fields, but this is more intensive than just filling a new table

In [ ]:
# Move the data into tables with the three new fields
try:
    db = mdb.connect(user=user,
                     passwd=password,
                     host=server,
                     port=port,
                     db=schema)
    
    cur = db.cursor()
    
    for table in ("test_data", "test_events", "train_data", "train_events"):
        # Get field names
        cur.execute("SHOW FIELDS IN %s" % table)
        fields = [row[0] for row in cur.fetchall()]

        # Create a new table mimicing the original ones
        cur.execute("CREATE TABLE temp_%s LIKE %s" % (table, table))
        
        # Add three fields
        cur.execute("""ALTER TABLE temp_%s
                       ADD `subject` INT AFTER id,
                       ADD `session` INT AFTER `subject`,
                       ADD `frame` INT AFTER `session`""" % table)
        
        # Create the string for inserting data from the original tables
        #   into the new ones
        insert_str = ("INSERT INTO temp_%s (" % table)
        for field in fields:
            insert_str += "`" + field + "`, "
        insert_str = insert_str[:-2] + ") SELECT * FROM %s"
        
        cur.execute(insert_str % table)
        
        # Get rid of the old ones
        cur.execute("DROP TABLE %s" % table)
        # Replace them with the new ones
        cur.execute("RENAME TABLE temp_%s TO %s" % (table, table))
            
    if db:
        db.close()

except mdb.Error, e:

    print "Error %d: %s" % (e.args[0], e.args[1])

### Populate the new fields by picking apart the 'id' field

In [ ]:
try:
    db = mdb.connect(user=user,
                     passwd=password,
                     host=server,
                     port=port,
                     db=schema)
    
    cur = db.cursor()
    
    for table in ("test_data", "test_events", "train_data", "train_events"):
        # Populate the fields
        cur.execute("""UPDATE %s SET 
                       `subject` = SUBSTR(id,5,LOCATE('_series',id)-5), 
                       `session` = SUBSTR(id,LOCATE('_series',id)+7,1),
                       `frame` = SUBSTRING_INDEX(id,"_",-1)""" % table)
    if db:
        db.close()

except mdb.Error, e:

    print "Error %d: %s" % (e.args[0], e.args[1])

### Add multi-key index on (subject, session, frame)

In [ ]:
try:
    db = mdb.connect(user=user,
                     passwd=password,
                     host=server,
                     port=port,
                     db=schema)
    
    cur = db.cursor()
    
    for table in ("test_data", "test_events", "train_data", "train_events"):
        # Populate the fields
        cur.execute("""CREATE UNIQUE INDEX subj_sess_frame
                       ON %s (`subject`, `session`, `frame`)
                       USING BTREE""" % table)
    if db:
        db.close()

except mdb.Error, e:

    print "Error %d: %s" % (e.args[0], e.args[1])

# Example data retrieval

In [ ]:
data_query = """
             SELECT * FROM %s_data
             WHERE subject=%i AND session=%i
             """

In [ ]:
events_query = """
               SELECT * FROM %s_events
               WHERE subject=%i AND session=%i
               """

In [ ]:
# For example, get data and events for subject 1, session 3
try:
    db = mdb.connect(user=user,
                     passwd=password,
                     host=server,
                     port=port,
                     db=schema)
    
    data_df = pd.read_sql(data_query % ("train", 1, 3), db)
    events_df = pd.read_sql(events_query % ("train", 1, 3), db)
    
    if db:
        db.close()

except mdb.Error, e:

    print "Error %d: %s" % (e.args[0], e.args[1])

In [ ]:
data_df.head(10)

In [ ]:
events_df.head(10)

# If you wanted all the data merged together...

In [ ]:
all_df = pd.merge(data_df, events_df, on=('id', 'subject', 'session', 'frame')); all_df.head(10)